In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import time
import os

# Current dir
current_dir = os.getcwd()

options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),  # Instalar el controlador automáticamente
    options=options
)
url = 'https://apps.bea.gov/itable/?ReqID=70&step=1&_gl=1*iib0n6*_ga*MjE0NTI5Mzk3NS4xNzQ1OTQxNTU4*_ga_J4698JNNFT*MTc0NTk0ODg3Mi4yLjEuMTc0NTk0ODkwOC4yNC4wLjA.#eyJhcHBpZCI6NzAsInN0ZXBzIjpbMSwyOSwyNSwzMSwyNiwyNywzMF0sImRhdGEiOltbIlRhYmxlSWQiLCI1MzIiXSxbIk1ham9yX0FyZWEiLCIwIl0sWyJTdGF0ZSIsWyIwIl1dLFsiQXJlYSIsWyJYWCJdXSxbIlN0YXRpc3RpYyIsWyIxIl1dLFsiVW5pdF9vZl9tZWFzdXJlIiwiTGV2ZWxzIl0sWyJZZWFyIixbIjIwMTYiLCIyMDE1IiwiMjAxNCIsIjIwMTMiLCIyMDEyIiwiMjAxMSJdXSxbIlllYXJCZWdpbiIsIi0xIl0sWyJZZWFyX0VuZCIsIi0xIl1dfQ=='

driver.get(url)

- Extraemos el PIB con Selenium de la tabla de GDP de 2011 a 2016

In [12]:
# Configuramos un objeto de espera con un tiempo máximo de 30 segundos.
# Esto nos permite esperar hasta que ciertas condiciones se cumplan en la página.
wait = WebDriverWait(driver, 30)

# Esperamos hasta que la tabla con ID 'DataTbl' sea visible en la página.
# Esto asegura que la tabla dinámica esté completamente renderizada antes de continuar.
table = wait.until(EC.visibility_of_element_located((By.ID, 'DataTbl')))

# Esperamos hasta que la instancia de DataTables para 'DataTbl' esté inicializada.
# Usamos JavaScript para verificar si DataTables está listo, asegurando que todo el contenido dinámico esté cargado.
wait.until(lambda driver: driver.execute_script("return typeof $('#DataTbl').DataTable() !== 'undefined';"))

# Usamos la API de DataTables para obtener todos los encabezados de las columnas.
# Ejecutamos JavaScript que accede a la tabla, obtiene los encabezados de las columnas y los convierte en una lista.
column_headers = driver.execute_script("""
var table = $('#DataTbl').DataTable();
return table.columns().header().map(function(th) {
    return $(th).text().trim();
}).toArray();
""")

# Buscamos el elemento <tbody> dentro de la tabla, que contiene las filas de datos.
tbody = table.find_element(By.TAG_NAME, 'tbody')

# Buscamos todos los elementos <tr> dentro del <tbody>, que representan las filas de la tabla.
rows = tbody.find_elements(By.TAG_NAME, 'tr')

# Inicializamos una lista vacía para almacenar los datos extraídos de las filas.
data = []

# Iteramos sobre cada fila de la tabla.
for row in rows:
    # Dentro de cada fila, buscamos todos los elementos <td>, que son las celdas de datos.
    cells = row.find_elements(By.TAG_NAME, 'td')
    # Extraemos el texto de cada celda, eliminando espacios en blanco adicionales, y lo añadimos a la lista 'data'.
    data.append([cell.text.strip() for cell in cells])

# Creamos un DataFrame de pandas usando los datos extraídos y los encabezados de las columnas obtenidos anteriormente.
pib = pd.DataFrame(data, columns=column_headers)

# Imprimimos las primeras filas del DataFrame para verificar que los datos se extrajeron correctamente.
print(pib.head())

# Cerramos la sesión del navegador para liberar recursos.
driver.quit()

  GeoFips        GeoName       2011:Q1       2011:Q2       2011:Q3  \
0   00000  United States  16,920,632.0  17,035,114.0  17,031,313.0   
1   01000        Alabama     199,677.1     200,443.7     201,353.8   
2   02000         Alaska      52,334.2      53,339.0      54,149.9   
3   04000        Arizona     287,340.2     288,496.1     289,990.1   
4   05000       Arkansas     114,337.2     114,769.4     115,128.4   

        2011:Q4       2012:Q1       2012:Q2       2012:Q3       2012:Q4  ...  \
0  17,222,583.0  17,367,010.0  17,444,525.0  17,469,650.0  17,489,852.0  ...   
1     204,016.8     204,204.3     205,005.6     202,857.9     202,634.3  ...   
2      56,358.6      56,280.5      57,869.0      57,662.9      56,537.0  ...   
3     295,818.7     296,784.1     297,961.7     296,789.8     296,773.3  ...   
4     116,732.0     117,016.9     117,033.7     116,535.8     115,878.2  ...   

        2014:Q3       2014:Q4       2015:Q1       2015:Q2       2015:Q3  \
0  18,406,941.0  18,500

- Verificamos las columnas obtenidas

In [13]:
pib.columns

Index(['GeoFips', 'GeoName', '2011:Q1', '2011:Q2', '2011:Q3', '2011:Q4',
       '2012:Q1', '2012:Q2', '2012:Q3', '2012:Q4', '2013:Q1', '2013:Q2',
       '2013:Q3', '2013:Q4', '2014:Q1', '2014:Q2', '2014:Q3', '2014:Q4',
       '2015:Q1', '2015:Q2', '2015:Q3', '2015:Q4', '2016:Q1', '2016:Q2',
       '2016:Q3', '2016:Q4'],
      dtype='object')

- Dropeamos la columna GeoFips para quedarnos con los valores que necesitamos

In [ ]:
pib.drop("GeoFips", axis = 1, inplace = True)

pib

,GeoName,2011:Q1,2011:Q2,2011:Q3,2011:Q4,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,...,2014:Q3,2014:Q4,2015:Q1,2015:Q2,2015:Q3,2015:Q4,2016:Q1,2016:Q2,2016:Q3,2016:Q4
0,United States,"16,920,632.0","17,035,114.0","17,031,313.0","17,222,583.0","17,367,010.0","17,444,525.0","17,469,650.0","17,489,852.0","17,662,400.0",...,"18,406,941.0","18,500,031.0","18,666,621.0","18,782,243.0","18,857,418.0","18,892,206.0","19,001,690.0","19,062,709.0","19,197,938.0","19,304,352.0"
1,Alabama,"199,677.1","200,443.7","201,353.8","204,016.8","204,204.3","205,005.6","202,857.9","202,634.3","207,096.6",...,"207,514.9","207,556.3","206,947.5","208,955.9","210,102.0","209,795.8","211,142.4","212,800.7","213,240.6","214,267.5"
2,Alaska,"52,334.2","53,339.0","54,149.9","56,358.6","56,280.5","57,869.0","57,662.9","56,537.0","56,001.0",...,"53,275.8","53,634.1","54,065.0","53,539.8","53,509.2","53,610.5","54,055.6","53,431.1","53,174.9","53,193.8"
3,Arizona,"287,340.2","288,496.1","289,990.1","295,818.7","296,784.1","297,961.7","296,789.8","296,773.3","298,558.9",...,"303,144.0","303,007.6","307,062.6","308,498.9","307,658.2","311,111.4","313,558.8","317,531.4","321,787.1","323,156.2"
4,Arkansas,"114,337.2","114,769.4","115,128.4","116,732.0","117,016.9","117,033.7","116,535.8","115,878.2","119,297.3",...,"121,859.3","122,126.9","120,865.2","121,200.5","121,989.6","122,074.8","122,538.2","122,691.9","122,814.1","124,095.2"
5,California,"2,185,033.7","2,194,927.2","2,189,999.3","2,201,588.5","2,231,663.5","2,242,371.4","2,256,039.1","2,291,675.1","2,287,676.1",...,"2,456,771.3","2,462,170.6","2,518,399.5","2,550,179.7","2,556,125.7","2,559,213.2","2,599,892.4","2,600,609.1","2,639,393.2","2,654,952.0"
6,Colorado,"282,496.8","283,488.4","283,865.5","286,004.4","287,339.1","289,500.3","288,999.0","291,519.8","298,232.7",...,"317,708.4","323,964.3","327,143.0","329,499.1","331,565.5","333,897.0","332,473.1","334,071.7","339,822.4","344,373.5"
7,Connecticut,"259,853.4","262,456.3","261,076.9","262,836.5","263,760.0","264,167.7","263,167.4","263,504.6","259,858.1",...,"262,121.8","265,935.7","268,284.2","269,967.0","268,581.2","268,646.6","268,037.6","267,474.9","267,530.8","269,741.5"
8,Delaware,"69,868.2","71,024.8","71,790.2","71,730.7","71,929.6","71,340.9","70,283.8","68,695.2","67,653.7",...,"75,297.4","75,553.6","76,773.7","77,932.7","75,399.8","74,738.5","72,505.8","72,358.7","71,532.1","71,369.1"
9,District of Columbia,"122,126.8","122,877.0","123,086.4","122,455.3","123,126.6","123,989.2","123,904.3","123,251.3","124,418.7",...,"127,427.3","127,953.1","128,720.2","129,520.7","128,927.9","129,272.0","131,617.9","132,312.1","132,029.2","132,911.9"


- Pasar las columnas Q1, Q2, Q3 y Q4 a meses

In [20]:
import re
# 1) Identificar columnas trimestrales
quarterly_cols = [
    col for col in pib.columns
    if re.match(r'^\d{4}:Q[1-4]$', col)
]

# 2) Cambiar a formato largo
df_long = pd.melt(
    pib,
    id_vars=['GeoName'],
    value_vars=quarterly_cols,
    var_name='Quarter',
    value_name='Quarterly_Income'
)

# 3) Convertir ingresos a numérico (quitando comas) y calcular ingreso mensual
df_long['Quarterly_Income'] = (
    df_long['Quarterly_Income']
      .astype(str)
      .str.replace(',', '', regex=False)
      .pipe(pd.to_numeric, errors='coerce')
)
df_long['Monthly_Income'] = df_long['Quarterly_Income'] / 3

# 4) Extraer año y trimestre
df_long[['Year','Q']] = df_long['Quarter'].str.split(':Q', expand=True)
df_long['Year'] = df_long['Year'].astype(int)
df_long['Q'] = 'Q' + df_long['Q']

# 5) Mapear cada trimestre a sus meses en español
months_map = {
    'Q1': ['enero', 'febrero', 'marzo'],
    'Q2': ['abril', 'mayo', 'junio'],
    'Q3': ['julio', 'agosto', 'septiembre'],
    'Q4': ['octubre', 'noviembre', 'diciembre']
}

# 6) Crear columna con lista de meses_año
df_long['Month'] = df_long.apply(
    lambda row: [f"{m}_{row['Year']}" for m in months_map[row['Q']]],
    axis=1
)

# 7) Explode para tener una fila por cada mes
df_monthly = df_long.explode('Month')[['GeoName','Month','Monthly_Income']]

# 8) Pivotar a formato ancho
pib_final = (
    df_monthly
      .pivot(index='GeoName', columns='Month', values='Monthly_Income')
      .reset_index()
)

# 9) Ordenar columnas cronológicamente usando nombres de meses en español
months = [
    'enero','febrero','marzo','abril','mayo','junio',
    'julio','agosto','septiembre','octubre','noviembre','diciembre'
]
month_order = {m: i+1 for i, m in enumerate(months)}

other_cols = [c for c in pib_final.columns if c != 'GeoName']
ordered = sorted(
    other_cols,
    key=lambda col: (
        int(col.split('_')[1]),           # año
        month_order[col.split('_')[0]]     # posición del mes
    )
)
pib_final = pib_final[['GeoName'] + ordered]


# 10) Mostrar

pib_final


Month,GeoName,enero_2011,febrero_2011,marzo_2011,abril_2011,mayo_2011,junio_2011,julio_2011,agosto_2011,septiembre_2011,...,marzo_2016,abril_2016,mayo_2016,junio_2016,julio_2016,agosto_2016,septiembre_2016,octubre_2016,noviembre_2016,diciembre_2016
0,Alabama,6.655903e+04,6.655903e+04,6.655903e+04,6.681457e+04,6.681457e+04,6.681457e+04,6.711793e+04,6.711793e+04,6.711793e+04,...,7.038080e+04,7.093357e+04,7.093357e+04,7.093357e+04,7.108020e+04,7.108020e+04,7.108020e+04,7.142250e+04,7.142250e+04,7.142250e+04
1,Alaska,1.744473e+04,1.744473e+04,1.744473e+04,1.777967e+04,1.777967e+04,1.777967e+04,1.804997e+04,1.804997e+04,1.804997e+04,...,1.801853e+04,1.781037e+04,1.781037e+04,1.781037e+04,1.772497e+04,1.772497e+04,1.772497e+04,1.773127e+04,1.773127e+04,1.773127e+04
2,Arizona,9.578007e+04,9.578007e+04,9.578007e+04,9.616537e+04,9.616537e+04,9.616537e+04,9.666337e+04,9.666337e+04,9.666337e+04,...,1.045196e+05,1.058438e+05,1.058438e+05,1.058438e+05,1.072624e+05,1.072624e+05,1.072624e+05,1.077187e+05,1.077187e+05,1.077187e+05
3,Arkansas,3.811240e+04,3.811240e+04,3.811240e+04,3.825647e+04,3.825647e+04,3.825647e+04,3.837613e+04,3.837613e+04,3.837613e+04,...,4.084607e+04,4.089730e+04,4.089730e+04,4.089730e+04,4.093803e+04,4.093803e+04,4.093803e+04,4.136507e+04,4.136507e+04,4.136507e+04
4,California,7.283446e+05,7.283446e+05,7.283446e+05,7.316424e+05,7.316424e+05,7.316424e+05,7.299998e+05,7.299998e+05,7.299998e+05,...,8.666308e+05,8.668697e+05,8.668697e+05,8.668697e+05,8.797977e+05,8.797977e+05,8.797977e+05,8.849840e+05,8.849840e+05,8.849840e+05
5,Colorado,9.416560e+04,9.416560e+04,9.416560e+04,9.449613e+04,9.449613e+04,9.449613e+04,9.462183e+04,9.462183e+04,9.462183e+04,...,1.108244e+05,1.113572e+05,1.113572e+05,1.113572e+05,1.132741e+05,1.132741e+05,1.132741e+05,1.147912e+05,1.147912e+05,1.147912e+05
6,Connecticut,8.661780e+04,8.661780e+04,8.661780e+04,8.748543e+04,8.748543e+04,8.748543e+04,8.702563e+04,8.702563e+04,8.702563e+04,...,8.934587e+04,8.915830e+04,8.915830e+04,8.915830e+04,8.917693e+04,8.917693e+04,8.917693e+04,8.991383e+04,8.991383e+04,8.991383e+04
7,Delaware,2.328940e+04,2.328940e+04,2.328940e+04,2.367493e+04,2.367493e+04,2.367493e+04,2.393007e+04,2.393007e+04,2.393007e+04,...,2.416860e+04,2.411957e+04,2.411957e+04,2.411957e+04,2.384403e+04,2.384403e+04,2.384403e+04,2.378970e+04,2.378970e+04,2.378970e+04
8,District of Columbia,4.070893e+04,4.070893e+04,4.070893e+04,4.095900e+04,4.095900e+04,4.095900e+04,4.102880e+04,4.102880e+04,4.102880e+04,...,4.387263e+04,4.410403e+04,4.410403e+04,4.410403e+04,4.400973e+04,4.400973e+04,4.400973e+04,4.430397e+04,4.430397e+04,4.430397e+04
9,Far West,1.016861e+06,1.016861e+06,1.016861e+06,1.022208e+06,1.022208e+06,1.022208e+06,1.020032e+06,1.020032e+06,1.020032e+06,...,1.198537e+06,1.199009e+06,1.199009e+06,1.199009e+06,1.215683e+06,1.215683e+06,1.215683e+06,1.225838e+06,1.225838e+06,1.225838e+06


- Renombra la columna de los estados para que coincida con el resto de DataFrames

In [21]:
pib_final.rename(columns={'GeoName': 'state_id'}, inplace=True)

In [22]:
## Guardar en CSV
# Path
pib_final_path = os.path.join(current_dir, '../../data/raw/macrodata')
# Crear un directorio si no existe
os.makedirs(pib_final_path, exist_ok=True)
# Archivo CSV
pib_final_file = os.path.join(pib_final_path, "pib_por_estados.csv")
# Guardar dataframe en CSV
pib_final.to_csv(pib_final_file, index=False)